In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
data.drop('CustomerID', axis=1, inplace=True)
data

In [ ]:
encoder = LabelEncoder()
data['Gender'] = encoder.fit_transform(data['Gender'])
mapped_class={i:v for i, v in enumerate(encoder.classes_)}
mapped_class
#pd.DataFrame(y)

In [ ]:
data

In [ ]:
profile = ProfileReport(data)
profile

In [ ]:
#binary 값들을 standard scaling 해도 괜찮은건지??

scaler = StandardScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)
scaled_data

# **Clustering**

In [ ]:
max_clusters = 50

In [ ]:
kmeans_tests = [KMeans(n_clusters=i, n_init=10) for i in range(1, max_clusters)]
inertias = [kmeans_tests[i].fit(scaled_data).inertia_ for i in range(len(kmeans_tests))]
inertias

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(range(1, max_clusters), inertias)
plt.xlabel('Number of clusters')
plt.ylabel('Inertias')
plt.title('Choosing number of clusters')
plt.show()

Choosing number of clusters 그래프에서 elbow method를 사용하여서 Number of clusters를 정한다.

위의 그래프에서 대략 8 ~ 9로 clusters의 개수를 잡으면 가장 efficient함을 알 수 있다.

Elbow Plot이라고도 불림

Inertias는 K개의 점을 기준으로 측정한 total variance라고 생각하면 편함.

In [ ]:
kmeans = KMeans(n_clusters=10, n_init=10)
kmeans.fit(scaled_data)

In [ ]:
clusters = kmeans.predict(scaled_data)
clusters

# **Using PCA to visualize the kmeans**

In [ ]:
pca = PCA(n_components=2)
reduced_data = pd.DataFrame(pca.fit_transform(scaled_data), columns=['PC1', 'PC2'])
reduced_data

In [ ]:
#4차원 공간에 존재하는 10개의 kmeans cluster centroid
kmeans.cluster_centers_

In [ ]:
#4차원 공간의 10개의 kmeans centroid를 pca를 통해서 2차원 공간의 점으로 바꾸기
#이미 fit_transform 했으므로
reduced_centers = pca.transform(kmeans.cluster_centers_)
reduced_centers

# **Data Visualization**

In [ ]:
reduced_data['cluster'] = clusters
reduced_data

In [ ]:
plt.figure(figsize=(14, 10))
colors=['red', 'blue', 'yellow', 'orange', 'cyan', 'magenta', 'brown', 'pink', 'green', 'purple']
for i in range(10):
    plt.scatter(reduced_data[reduced_data['cluster']==i].loc[:,'PC1'], reduced_data[reduced_data['cluster']==i].loc[:,'PC2'], color=colors[i])

plt.scatter(reduced_centers[:, 0], reduced_centers[:,1], color='black', marker='x', s=300)

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
#THE END!!